<a href="https://colab.research.google.com/github/eisbetterthanpi/transformer/blob/main/translation_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title setup

# https://pytorch.org/tutorials/beginner/translation_transformer.html
# https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/c64c91cf87c13c0e83586b8e66e4d74e/translation_transformer.ipynb

# https://github.com/pytorch/data
%pip install portalocker
%pip install torchdata

# Create source and target language tokenizer. Make sure to install the dependencies.
!pip install -U torchdata
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm


In [ ]:
# @title data

from torchtext.datasets import multi30k, Multi30k
# modify the URLs for the dataset since the links to the original dataset are broken https://github.com/pytorch/text/issues/1756#issuecomment-1163664163
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de'
TRG_LANGUAGE = 'en'

from torchtext.data.utils import get_tokenizer
de_tokenizer = get_tokenizer('spacy', language='de_core_news_sm')
en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')


UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3 # unknown, pad, bigining, end of sentence
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

from torchtext.vocab import build_vocab_from_iterator
train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TRG_LANGUAGE))

de_tokens = [de_tokenizer(data_sample[0]) for data_sample in train_iter]
en_tokens = [en_tokenizer(data_sample[1]) for data_sample in train_iter]

de_vocab = build_vocab_from_iterator(de_tokens, min_freq=1, specials=special_symbols, special_first=True)
en_vocab = build_vocab_from_iterator(en_tokens, min_freq=1, specials=special_symbols, special_first=True)
de_vocab.set_default_index(UNK_IDX)
en_vocab.set_default_index(UNK_IDX)

import torch

def de_transform(o):
    o=de_tokenizer(o)
    o=de_vocab(o)
    return torch.cat((torch.tensor([BOS_IDX]), torch.tensor(o), torch.tensor([EOS_IDX])))

def en_transform(o):
    o=en_tokenizer(o)
    o=en_vocab(o)
    return torch.cat((torch.tensor([BOS_IDX]), torch.tensor(o), torch.tensor([EOS_IDX])))


from torch.nn.utils.rnn import pad_sequence
# function to collate data samples into batch tensors
def collate_fn(batch): # convert a batch of raw strings into batch tensors
    src_batch, trg_batch = [], []
    for src_sample, trg_sample in batch:
        src_batch.append(de_transform(src_sample.rstrip("\n")))
        trg_batch.append(en_transform(trg_sample.rstrip("\n")))
    # src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    # trg_batch = pad_sequence(trg_batch, padding_value=PAD_IDX)
    src_batch = pad_sequence(src_batch, batch_first=True, padding_value=PAD_IDX)
    trg_batch = pad_sequence(trg_batch, batch_first=True, padding_value=PAD_IDX)
    return src_batch, trg_batch


torch.manual_seed(0)

train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TRG_LANGUAGE))
val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TRG_LANGUAGE))
batch_size = 128 # 128
train_loader = torch.utils.data.DataLoader(train_iter, batch_size=batch_size, collate_fn=collate_fn)
val_loader = torch.utils.data.DataLoader(val_iter, batch_size=batch_size, collate_fn=collate_fn)

# vocab_transform = {SRC_LANGUAGE:de_vocab, TRG_LANGUAGE:en_vocab}
# text_transform = {SRC_LANGUAGE:de_transform, TRG_LANGUAGE:en_transform}


In [10]:
# @title model
import torch
import torch.nn as nn
import math
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class PositionalEncoding(nn.Module):
    def __init__(self, emb_size, dropout, maxlen = 128): # maxlen=5000
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(dropout)
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1) # .reshape(-1, 1)
        pos_emb = torch.zeros((maxlen, emb_size))
        pos_emb[:, 0::2] = torch.sin(pos * den) # PE(pos, 2i) = sin(pos/1000^(2i/dim_model))
        pos_emb[:, 1::2] = torch.cos(pos * den) # PE(pos, 2i + 1) = cos(pos/1000^(2i/dim_model))
        pos_emb = pos_emb.unsqueeze(0) # batch_first=F -> unsqueeze(-2)
        self.register_buffer('pos_emb', pos_emb) # register as buffer so optimizer wont update it

    def forward(self, token_emb):
        return self.dropout(token_emb + self.pos_emb[:, :token_emb.size(1)]) # batch_first=F -> [:token_emb.size(0), :]


class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)


class Transformer(nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, d_model=512, nhead=8, num_encoder_layers=6, num_decoder_layers=6, dim_feedforward=2048, dropout=0.1):
        super(Transformer, self).__init__()
        self.d_model = d_model
        # self.src_tok_emb = nn.Embedding(src_vocab_size, emb_size)
        # self.trg_tok_emb = nn.Embedding(trg_vocab_size, emb_size)
        self.pos_enc = PositionalEncoding(d_model, dropout=dropout)
        self.transformer = nn.Transformer(d_model=d_model, nhead=nhead, num_encoder_layers=num_encoder_layers, num_decoder_layers=num_decoder_layers, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True)
        self.generator = nn.Linear(d_model, trg_vocab_size)

        self.src_tok_emb = TokenEmbedding(src_vocab_size, d_model)
        self.trg_tok_emb = TokenEmbedding(trg_vocab_size, d_model)
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, src, trg, src_mask=None, trg_mask=None, src_key_padding_mask=None, trg_key_padding_mask=None):
        src_emb = self.pos_enc(self.src_tok_emb(src))
        trg_emb = self.pos_enc(self.trg_tok_emb(trg))

        out = self.transformer(src_emb, trg_emb, src_mask=src_mask, tgt_mask=trg_mask, memory_mask=None, src_key_padding_mask=src_key_padding_mask, tgt_key_padding_mask=trg_key_padding_mask, memory_key_padding_mask=src_key_padding_mask)
        return self.generator(out)

    def encode(self, src, src_mask=None):
        return self.transformer.encoder(self.pos_enc(self.src_tok_emb(src)), src_mask)

    def decode(self, trg, memory, trg_mask=None, memory_mask=None):
        return self.transformer.decoder(self.pos_enc(self.trg_tok_emb(trg)), memory, trg_mask, memory_mask)


src_vocab_size = len(de_vocab)
trg_vocab_size = len(en_vocab)

emb_size = 512 # d_model
nhead = 8
dim_feedforward = 512
num_encoder_layers = 3
num_decoder_layers = 3

model = Transformer(src_vocab_size, trg_vocab_size, d_model=512, nhead=8, num_encoder_layers=3, num_decoder_layers=3, dim_feedforward=512, dropout=0.1).to(device)


In [11]:
# @title mask translate

# subsequent word mask that will prevent the model from looking into the future words when making predictions.
# also need masks to hide source and target padding token
def generate_square_subsequent_mask(sz):
    mask = torch.tril(torch.ones((sz, sz), device=device))#.bool()
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, trg):
    src_seq_len = src.shape[1] # batch_first=F -> [0]
    trg_seq_len = trg.shape[1] # batch_first=F -> [0]
    trg_mask = generate_square_subsequent_mask(trg_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=device).type(torch.bool)
    src_padding_mask = (src == PAD_IDX) # batch_first=F -> .transpose(0, 1)
    trg_padding_mask = (trg == PAD_IDX) # batch_first=F -> .transpose(0, 1)
    return src_mask, trg_mask, src_padding_mask, trg_padding_mask


def translate(model, src_sentence):
    model.eval()
    src = de_transform(src_sentence).view(1,-1).to(device)
    num_tokens = src.shape[1] # batch_first=F -> [0]
    src_mask = torch.zeros((num_tokens, num_tokens), dtype=bool, device=device)
    trg_indexes = [BOS_IDX]
    max_len = src.shape[1]+5
    for i in range(max_len):
        trg_tensor = torch.tensor(trg_indexes, dtype=torch.long, device=device).unsqueeze(0)
        trg_mask = generate_square_subsequent_mask(trg_tensor.size(1))
        with torch.no_grad():
            output = model(src, trg_tensor, src_mask, trg_mask)
        pred_token = output.argmax(2)[:,-1].item() # batch_first=F -> ?
        trg_indexes.append(pred_token)
        if pred_token == EOS_IDX: break
    trg_tokens = torch.tensor(trg_indexes[1:-1]).flatten()
    return " ".join(en_vocab.lookup_tokens(list(trg_tokens.cpu().numpy())))

# UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3 # unknown, pad, bigining, end of sentence
print(translate(model, "Eine Gruppe von Menschen steht vor einem Iglu ."))


coca technique Toy technique Toy technique stickers watch Toy stickers technique stickers technique stickers technique


In [12]:
# @title train test

def train(dataloader, model, loss_fn, optimizer):
    model.train()
    losses = 0
    for src, trg in dataloader:
        src = src.to(device)
        trg = trg.to(device)
        trg_input = trg[:, :-1] # batch_first=F -> [:-1, :]
        src_mask, trg_mask, src_padding_mask, trg_padding_mask = create_mask(src, trg_input)
        logits = model(src, trg_input, src_mask, trg_mask, src_padding_mask, trg_padding_mask)
        optimizer.zero_grad()
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), trg[:, 1:].reshape(-1)) # batch_first=F -> trg[1:, :]
        loss.backward()
        optimizer.step()
        losses += loss.item()
    return losses / len(list(dataloader))

def test(dataloader, model, loss_fn):
    model.eval()
    losses = 0
    for src, trg in dataloader:
        src = src.to(device)
        trg = trg.to(device)
        trg_input = trg[:, :-1] # batch_first=F -> [:-1, :]
        src_mask, trg_mask, src_padding_mask, trg_padding_mask = create_mask(src, trg_input)
        logits = model(src, trg_input, src_mask, trg_mask, src_padding_mask, trg_padding_mask)
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), trg[:, 1:].reshape(-1)) # batch_first=F -> trg[1:, :]
        losses += loss.item()
    return losses / len(list(dataloader))


In [13]:
# @title run
import time

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9) # lr=0.0001

for epoch in range(18):
    start_time = time.time()
    train_loss = train(train_loader, model, loss_fn, optimizer)
    end_time = time.time()
    val_loss = test(val_loader, model, loss_fn)
    print((f"Epoch: {epoch+1}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    print(translate(model, "Eine Gruppe von Menschen steht vor einem Iglu ."))


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/datapipes/iter/combining.py:333: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


Epoch: 1, Train loss: 5.338, Val loss: 4.101, Epoch time = 45.662s
A group of people are in a red shirt is playing a large
Epoch: 2, Train loss: 3.766, Val loss: 3.332, Epoch time = 45.664s
A group of people are standing in front of a crowd .
Epoch: 3, Train loss: 3.165, Val loss: 2.891, Epoch time = 45.248s
A group of people standing in front of a crowd .
Epoch: 4, Train loss: 2.768, Val loss: 2.630, Epoch time = 45.014s
A group of people standing in front of a store .
Epoch: 5, Train loss: 2.477, Val loss: 2.453, Epoch time = 46.618s
A group of people standing in front of a store .
Epoch: 6, Train loss: 2.246, Val loss: 2.318, Epoch time = 44.933s
A group of people standing in front of a store .
Epoch: 7, Train loss: 2.052, Val loss: 2.203, Epoch time = 45.686s
A group of people standing in front of a wooden computer .
Epoch: 8, Train loss: 1.892, Val loss: 2.115, Epoch time = 45.169s
A group of people standing in front of an open computer .
Epoch: 9, Train loss: 1.753, Val loss: 2.0

In [14]:
# @title inference
print(translate(model, "Eine Gruppe von Menschen steht vor einem Iglu .")) # A group of people stand in front of an igloo .
print(translate(model, "Ein Koch in weißer Uniform bereitet Essen in einer Restaurantküche zu .")) # A chef in a white uniform prepares food in a restaurant kitchen .
print(translate(model, "Zwei junge Mädchen spielen Fußball auf einem Feld. .")) # Two young girls play soccer on a field. .
print(translate(model, "Eine Frau mit Hut und Sonnenbrille steht am Strand .")) # A woman wearing a hat and sunglasses stands on the beach .
print(translate(model, "Zwei Freunde lachen und genießen ein Eis auf einer wunderschönen Wiese .")) # Two friends laugh and enjoy ice cream on a beautiful meadow .


A group of people stand in front of an igloo
A chef in a white uniform prepares food in a restaurant kitchen .
Two young girls play soccer on a field .
A woman wearing a hat and sunglasses is standing on the beach .
Two friends laugh and enjoy an ice on a beautiful field .
